IMAGE CLASSIFICATION WITH TENSOR FLOW AND OPEN CV... Custom model is used ;) 


0 left,
1 right

In [1]:
import tensorflow
from PIL import Image
import numpy as np

In [2]:
import socket

In [3]:
import cv2
import time

In [4]:
#SOCKET INIT

# host = '192.168.43.3' #Raspi address
# port = 3032 #communication port
# sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# sock.connect((host, port))

def close_socket():
    sock.close()
    
def send_str_socket(string):
    command = string
    #sock.send(command.encode('utf-8'))
    

In [5]:
# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

In [6]:
#load the model
model = tensorflow.keras.models.load_model('keras_model.h5', compile=False)

In [7]:
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

In [8]:
#FUNCTION TO OVERLAY TEXT ON Cv2 Video Capture

def print_cv2(text, x, y, b, g, r):
    cv2.putText(image_feed, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (b, g, r))

In [9]:
#TRIGGERS FOR SOCKET

class0 = False
class1 = False
class2 = False
class3 = False

In [10]:
#ALL THE CONDITIONS BASED ON THE PREDICTIONS
#[0, X] where X is the number of classes in the model


def prediction_conditions():
    
    global class0, class1, class2, class3
    
    #BOTTOM BOX FOR PREDICTION TEXT
    cv2.rectangle(image_feed, (5, 440), (430, 470), (0, 0, 0), -1)
    
    #TOP BOX FOR OUTCOME
    cv2.rectangle(image_feed, (5, 10), (80, 40), (255, 255, 255), -1)
    
    print_cv2(str(prediction[0]), 10, 460, 255, 255, 255)
    
    if ((prediction[0, 0] > prediction[0, 1]) and (prediction[0, 0] > prediction[0, 2]) and (prediction[0, 0] > prediction[0, 3])):
        cv2.rectangle (image_feed, (0, 0), (640, 480), (0, 255, 0), 5) #FIRST CONDITION, green color
        print_cv2("Up", 10, 30, 0, 255, 0)
        if class0 == False:
            send_str_socket('up')
            class0 = True
            class1 = False
            class2 = False
            class3 = False
     
    elif ((prediction[0, 1] > prediction[0, 0]) and (prediction[0, 1] > prediction[0, 2]) and (prediction[0, 1] > prediction[0, 3])):
        cv2.rectangle (image_feed, (0, 0), (640, 480), (255, 0, 0), 5) #SECOND CONDITION, blue color
        print_cv2("Down", 10, 30, 255, 0, 0)
        if class1 == False:
            send_str_socket('down')
            class0 = False
            class1 = True
            class2 = False
            class3 = False
        
    elif ((prediction[0, 2] > prediction[0, 0]) and (prediction[0, 2] > prediction[0, 1]) and (prediction[0, 2] > prediction[0, 3])):
        cv2.rectangle (image_feed, (0, 0), (640, 480), (0, 0, 255), 5) #THIRD CONDITION, red color 
        print_cv2("Left", 10, 30, 0, 0, 255)
        if class2 == False:
            send_str_socket('left')
            class0 = False
            class1 = False
            class2 = True
            class3 = False
        
    elif ((prediction[0, 3] > prediction[0, 0]) and (prediction[0, 3] > prediction[0, 1]) and (prediction[0, 3] > prediction[0, 2])):
        cv2.rectangle (image_feed, (0, 0), (640, 480), (0, 255, 255), 5) # FOURTH COMDITION, yellow color
        print_cv2("Right", 10, 30, 0, 255, 255)
        if class3 == False:
            send_str_socket('right')
            class0 = False
            class1 = False
            class2 = False
            class3 = True

In [12]:
#listen onto the droid cam video feed.
#Use cv2.VideoCapture(0) to use wired usb camera. Where 0 is the index of the camera
cap = cv2.VideoCapture(0)

#Main loop
while True:
    
    ret, image_feed = cap.read()
    
    image_feed = cv2.flip(image_feed, 1) #Flip the video horizontally
    
    M = cv2.getRotationMatrix2D( (320, 240), 270, 1.42)
    rotated_image = cv2.warpAffine( image_feed, M, (480, 640))
    
    #cv2.imshow("Output 0, raw", image_feed)
    
    #tik = time.time()
    
    cv2.imwrite('pics\output.jpg', rotated_image)
    
    image = Image.open('pics\output.jpg')
    
    #tok = time.time()
    
    image = image.resize((224, 224))
    image_array = np.asarray(image)
    normalized_image_array = image_array / 255.0
    data[0] = normalized_image_array
    prediction = model.predict(data)
    
    #tuk = time.time()
    
    #PRINT THE TIME DIFFERENCE BETWEEN FUNCTIONS
    '''print("Time to save and read image: " + str(tok - tik))
    print("Time to classify image: " + str(tuk - tok))
    print("Total time wasted: " + str((tok - tik) + (tuk - tok)))'''
    
    prediction_conditions()
    
    #cv2.imshow('OUTPUT 0', flip_feed)
    cv2.imshow('output 0, raw', rotated_image)
    
    if cv2.waitKey(1) & 0xff == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [56]:
close_socket()